# Setear proyecto

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
import xml.dom.minidom
import numpy as np

In [ ]:
from google.colab import files

In [ ]:
#Instalar graficos
!pip install scikit-plot

In [ ]:
cd /content/drive/My\ Drive/IA_project/test

In [ ]:
#Inicializar
path="/content/drive/My Drive/IA_project/labeled/"
size_photo=1000



---



# **Cargar archivos labeling**

## Imagen Row

Recorta la imagen a partir del xml. Estandarisa a resolucion 400x108 y la convierte a monocromatico


In [ ]:
def standardize_img(foto):
  foto_tem = foto.resize((400,108),Image.ANTIALIAS)
  foto_tem= foto_tem.convert('L') #Monochromatic
  return foto_tem


In [ ]:
def process_img_row(ele_name):
    #IMG Row
    fotoRaw = Image.open(path+"img/"+ele_name+".jpg")
    xmlRaw = xml.dom.minidom.parse(path+"label/"+ele_name+".xml");

    xmin = int(xmlRaw.getElementsByTagName("xmin")[0].firstChild.data)
    ymin = int(xmlRaw.getElementsByTagName("ymin")[0].firstChild.data)
    xmax = int(xmlRaw.getElementsByTagName("xmax")[0].firstChild.data)
    ymax = int(xmlRaw.getElementsByTagName("ymax")[0].firstChild.data)

    
    if(False):# Label
      width = int(xmlRaw.getElementsByTagName("width")[0].firstChild.data)
      height = int(xmlRaw.getElementsByTagName("height")[0].firstChild.data)
      xmin = (xmin*size_photo)//width
      xmax = (xmax*size_photo)//width
      ymin = (ymin*size_photo)//height
      ymax = (ymax*size_photo)//height
      fotoTest = fotoRaw.resize((size_photo,size_photo),Image.ANTIALIAS)
      fotoTest= fotoTest.convert('L')#L  F
      #save_file_png(fotoTest,ele_name,"fotoRaw")
      fotoTest_lis = list(fotoTest.getdata())
      mat_foto.append(fotoTest_lis)
      mat_res_foto.append((xmin,ymin))
      area_row = (xmin, ymin, xmax, ymax)
      foto = fotoTest.crop(area_row)
      
    # Crop 
    area_row = (xmin, ymin, xmax, ymax)
    foto = fotoRaw.crop(area_row)

    #Standardize
    foto = standardize_img(foto)
    
    #Close
    fotoRaw.close()

    return foto

## Prosesar caracteres

Recorta la imagen en tres partes que luego retorna

In [ ]:
def process_char(foto):
    a1 = (0, 0, foto.size[0]//3, foto.size[1])
    a2 = (foto.size[0]//3, 0, foto.size[0]-foto.size[0]//3, foto.size[1])
    a3 = (foto.size[0]-foto.size[0]//3, 0, foto.size[0], foto.size[1])

    c1 = foto.crop(a1)
    c2 = foto.crop(a2)
    c3 = foto.crop(a3)

    return (c1,c2,c3)

## Save Files

Guardar imagenes en directorio ``` test_img/ ```



In [ ]:
def save_file_png(img,name,path_tem):
  path_img_result = 'test_img/'
  path_img_result_end = path_img_result+path_tem

  if not os.path.isdir(path_img_result):
    print('La carpeta no existe 1.');
    os.mkdir(path_img_result)

  if not os.path.isdir(path_img_result_end):
    print('La carpeta no existe 2.');
    os.mkdir(path_img_result_end)

  img.save(path_img_result_end+"/"+name+'.png')

## Carga de archivos etiquetados

Carga todos los archivos en el directorio ``` img/ ``` con sus respectivos ``` .xml ``` 

Los prosesa hasta tener cuatro listas:

*   Lista de pixeles (0,256) de cada numero
*   Numeros esperados
*   Lista de pixeles (0,256) de cada letra
*   Letras esperados






In [ ]:
def load_labeling(download_state):
  contenido = os.listdir(path+"img")
  print(contenido)
  count = 0
  mat_leter = []; 
  mat_res_leter = []
  mat_digit = []; 
  mat_res_digit = []
  for ele in contenido:
    count=count+1
    print(str(count)+" de " + str(len(contenido)))
    ele_name =ele.split(".")[0]

    foto = process_img_row(ele_name)

    # Crop digit and leter
    area_leter = (0, 0, (foto.size[0]//2)-23, foto.size[1])
    area_digit = ((foto.size[0]//2)+17, 0, foto.size[0], foto.size[1])
    foto_leter = foto.crop(area_leter)
    foto_digit = foto.crop(area_digit)

    # Leter
    (l1,l2,l3)=process_char(foto_leter)

    mat_leter.append(list(l1.getdata()))
    mat_leter.append(list(l2.getdata()))
    mat_leter.append(list(l3.getdata()))

    mat_res_leter.append(ord(ele[0]))
    mat_res_leter.append(ord(ele[1]))
    mat_res_leter.append(ord(ele[2]))

    ## Digito
    (d1,d2,d3)=process_char(foto_digit)

    mat_digit.append(list(d1.getdata()))
    mat_digit.append(list(d2.getdata()))
    mat_digit.append(list(d3.getdata()))

    mat_res_digit.append(int(ele[3]))
    mat_res_digit.append(int(ele[4]))
    mat_res_digit.append(int(ele[5]))

    #Download
    if(download_state):
      save_file_png(foto,ele_name,"foto")

      save_file_png(l1,ele[0]+"_0_"+ele_name,"l")
      save_file_png(l2,ele[1]+"_1_"+ele_name,"l")
      save_file_png(l3,ele[2]+"_2_"+ele_name,"l")

      save_file_png(d1,ele[3]+"_0_"+ele_name,"d")
      save_file_png(d2,ele[4]+"_1_"+ele_name,"d")
      save_file_png(d3,ele[5]+"_2_"+ele_name,"d")

  print("End")
  return (mat_leter,mat_res_leter,mat_digit,mat_res_digit)



---



# **Entrenamiento**

## Split

Parte las listas en train y test siendo test del 20%

In [ ]:
from sklearn.model_selection import train_test_split
def split(mat,mat_res):
  X_train, X_test, y_train, y_test = train_test_split(mat, mat_res, test_size=0.20)
  return(X_train, X_test, y_train, y_test)
  print("End")

## Persistencia

http://exponentis.es/persistencia-de-modelos-en-python-como-guardar-tu-modelo-entrenado-de-machine-learning

In [ ]:
import joblib 
def saveModel(clf_d_save,name):
  joblib.dump(clf_d_save, name+'.pkl') # Guardo el modelo.

def openModel(name):
  clf_d_save = joblib.load(name+'.pkl') # Carga del modelo.
  return clf_d_save

## **Modelos**

 ### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

def train_RF():
  print("Prosesando...")
  #,criterion='entropy'
  #(n_estimators=50000, random_state=0,n_jobs=-1,criterion='entropy')
  #Digit
  clf_l = RandomForestClassifier(n_estimators=1000, random_state=0,n_jobs=-1)
  clf_l.fit(X_train_l, y_train_l)

  #Number
  clf_d = RandomForestClassifier(n_estimators=50000, random_state=0,n_jobs=-1,criterion='entropy')
  clf_d.fit(X_train_d, y_train_d)

  print("Fin")
  return (clf_l,clf_d)

### Simple vector machine

In [ ]:
#https://scikit-learn.org/stable/modules/svm.html#classification
#multi_class='crammer_singer'
#clf = svm.SVC()

In [ ]:
from sklearn import svm
def train_LinSVM():
  print("Prosesando...")

  #Digit
  clf_l = svm.LinearSVC(max_iter=5000,multi_class='ovr')
  clf_l.fit(X_train_l, y_train_l)

  #Number
  clf_d = svm.LinearSVC(max_iter=5000,multi_class='ovr')
  clf_d.fit(X_train_d, y_train_d)

  print("Fin")
  return (clf_l,clf_d)

def train_SVM():
  print("Prosesando...")
  #Digit
  clf_l = svm.SVC(max_iter=-1, kernel='poly',decision_function_shape='ovr',cache_size=800,degree=1  ,gamma='auto')
  clf_l.fit(X_train_l, y_train_l)

  #Number
  clf_d = svm.SVC(max_iter=-1, kernel='poly',decision_function_shape='ovr',cache_size=800,degree=1  ,gamma='auto')
  clf_d.fit(X_train_d, y_train_d)

  print("Fin")
  return (clf_l,clf_d)


### Decision tree

In [ ]:
from sklearn import tree

def train_tree():
  print("Prosesando...")

  #Digit
  clf_l = tree.DecisionTreeClassifier()
  clf_l.fit(X_train_l, y_train_l)

  #Number
  clf_d = tree.DecisionTreeClassifier()
  clf_d.fit(X_train_d, y_train_d)

  print("Fin")
  return (clf_l,clf_d)

### Stochastic Gradient Descent

https://scikit-learn.org/stable/modules/sgd.html#classification


In [ ]:
from sklearn.linear_model import SGDClassifier
def train_SGD():
  print("Prosesando...")
  #Digit
  clf_l =SGDClassifier(loss="hinge", penalty="l2", max_iter=500)
  clf_l.fit(X_train_l, y_train_l)

  #Number
  clf_d = SGDClassifier(loss="hinge", penalty="l2", max_iter=500)
  clf_d.fit(X_train_d, y_train_d)

  print("Fin")
  return (clf_l,clf_d)

### Neighbors

In [ ]:
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier

def train_nearest_centroid():
  print("Prosesando...")

  #Digit
  clf_l =NearestCentroid()
  clf_l.fit(X_train_l, y_train_l)

  #Number
  clf_d = NearestCentroid()
  clf_d.fit(X_train_d, y_train_d)

  print("Fin")
  return (clf_l,clf_d)


def train_k_neighbors():
  print("Prosesando...")
  #Digit
  clf_l =KNeighborsClassifier(n_neighbors=10)
  clf_l.fit(X_train_l, y_train_l)

  #Number
  clf_d = KNeighborsClassifier(n_neighbors=10)
  clf_d.fit(X_train_d, y_train_d)

  print("Fin")
  return (clf_l,clf_d)



---



# **Testeo**

**Test dataset**

In [ ]:
from sklearn import metrics


def analyzeT(y_pred_l,y_pred_d):
  #Letra
  print("---------- Letra ----------")
  analyze_aux(y_pred_l,y_test_l)

  #Digito
  print("---------- Digito ----------")
  analyze_aux(y_pred_d,y_test_d)



def analyze_aux(y_pred,y_test):
  print(metrics.classification_report(y_test, y_pred))



**Test img**

In [ ]:
def test_img(name):
  foto2 = Image.open("/content/drive/My Drive/IA_project/ej/"+name+".jpg")

  foto2=standardize_img(foto2) 

  mat_leter=[]
  mat_digit=[]
  # Recorte

  foto_leter = foto2.crop(area_leter)
  foto_digit = foto2.crop(area_digit)


  # Leter
  (l1,l2,l3)=process_char(foto_leter)

  mat_leter.append(list(l1.getdata()))
  mat_leter.append(list(l2.getdata()))
  mat_leter.append(list(l3.getdata()))

  ## Digito
  (d1,d2,d3)=process_char(foto_digit)

  mat_digit.append(list(d1.getdata()))
  mat_digit.append(list(d2.getdata()))
  mat_digit.append(list(d3.getdata()))


  res_l= clf_l.predict(mat_leter)
  res_d= clf_d.predict(mat_digit)
  for element in res_l:
    print(" caracter: " +chr(element))

  for element in res_d:
    print(" numero: " +str(element))





# **Ejecutar**

In [ ]:
#Load
(mat_leter,mat_res_leter,mat_digit,mat_res_digit) = load_labeling(True)

In [ ]:
#Split
(X_train_l, X_test_l, y_train_l, y_test_l ) = split(mat_leter,mat_res_leter)
(X_train_d, X_test_d, y_train_d, y_test_d ) = split(mat_digit,mat_res_digit)

In [ ]:
#Train
(clf_l,clf_d)=train_SVM()

In [ ]:
y_pred_l = clf_l.predict(X_test_l)
y_pred_d = clf_d.predict(X_test_d)

analyzeT(y_pred_l,y_pred_d)

In [ ]:
import scikitplot as skplt

skplt.metrics.plot_confusion_matrix(y_test_l, y_pred_l,figsize =(15,15),normalize=False)

plt.show()

### Cross validation


In [ ]:
from sklearn.model_selection import cross_val_predict
import os


clf_test6 = NearestCentroid()
clf_test5 = KNeighborsClassifier(n_neighbors=40)#n_neighbors=10,n_jobs=-1
clf_test4 = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
clf_test3 = svm.SVC(max_iter=500)
clf_test2 = svm.LinearSVC(max_iter=1000,multi_class='ovr')#ovr  crammer_singer  loss="hinge" penalty="l1" 1000
clf_test1 = RandomForestClassifier(n_estimators=50000, random_state=0,n_jobs=-1,criterion='entropy')
clf_test0 = RandomForestClassifier(n_estimators=1000, random_state=0,n_jobs=-1,criterion='entropy')
#clf_test = svm.LinearSVC(max_iter=(i*multipicador),multi_class='crammer_singer')#crammer_singer

def cross_validation(clf,train,test):
  predictions = cross_val_predict(clf, train, test,n_jobs=-1,cv= 4)
  analyze_aux(predictions,test)
  #skplt.metrics.plot_confusion_matrix(mat_res_leter, predictions,figsize =(15,15),normalize=False)
  #plt.show()
  return predictions

def csv_create(name,acurray,num_iter):
  file = open(name+".csv", "w")
  file.write("max_iter ; Accuracy ;" +'\n')
  for i in range(0,len(acurray)):
    file.write(str(num_iter*(i+1))+";"+acurray[i]+";" +'\n')       
  file.close();


def analyze(name,multipicador,train,test):
  acurray=[]
  for i in range (1,10):#10
    print("-------------------------------------- "+str(i+multipicador)+" -------------------------------------- ")
    clf_test = svm.SVC(max_iter=-1, kernel='rbf',decision_function_shape='ovr',cache_size=800,degree=1  ,gamma='auto', tol=0.1, C=0.1, probability=True)#crammer_singer  ovr  tol=0.0001, C=1.0   poly rbf sigmoid degree=1 
    #clf_test =NearestCentroid()
    y_pred=cross_validation(clf_test,train,test)
    y_test=test
    acurray.append(str(metrics.accuracy_score(y_test, y_pred)).replace(".", ","))
  csv_create(name,acurray,(multipicador))


set_test=mat_leter
set_res=mat_res_leter
set_name="none20_l"
for i in range (0,5): #10
  analyze(set_name+"_"+str(i),10**i,set_test,set_res)

set_test=mat_digit
set_res=mat_res_digit
set_name="none20_d"
for i in range (0,5):
  analyze(set_name+"_"+str(i),10**i,set_test,set_res)
